<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#What-is-ROS-?" data-toc-modified-id="What-is-ROS-?-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>What is ROS ?</a></span></li><li><span><a href="#How-does-ROS-work-?" data-toc-modified-id="How-does-ROS-work-?-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>How does ROS work ?</a></span><ul class="toc-item"><li><span><a href="#ROS-master" data-toc-modified-id="ROS-master-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>ROS master</a></span></li></ul></li><li><span><a href="#Why-ROS-?" data-toc-modified-id="Why-ROS-?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Why ROS ?</a></span></li><li><span><a href="#Sources:" data-toc-modified-id="Sources:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sources:</a></span></li></ul></div>

<img src="images/ros_logo.png" width="200" />

# What is ROS ?

ROS is an abbreviation for '*Robot Operating System*' and is **open source** robotics middleware, which means it is a collection of software frameworks for robotic software development. 
It provides the services you would expect from an operating system, including hardware abstraction, low-level device control, implementation of commonly-used functionality, message-passing between processes, and package management which can be used to create robot applications.

The primary goal of ROS is to support the reuse of code. ROS is a distributed framework of processes that enables executables to be individually designed and loosely coupled. These processes are called **Nodes** and can be grouped into **Packages** and **Stacks**. This makes sharing and distributing code easy.
ROS also supports a system of code **Repositories** to collaborate on projects and distribute them. Commonly used is Github to work on or clone full packages. This allows you to easily manage and maintain your projects and their development.
 
We can look at ROS of having 5 practical facets:
<img src="images/ros_facets.png" width="500" />

ROS provides modular software development by providing a library of free and reusable code packages. This makes robot development faster,easier and more reliable.
It provides a Run-Time Environment which supports near Real-Time communication between system elements and data sharing. 
ROS itself is not a strict standard of programming, but it provides conventions for creating and using ROS supported code in a repeatable and reliable way. For instance, ROS encourages to use the same data message structure to send data between different modules. 
It also provides development tools for monitoring, visualizing and troubleshooting your robot. This allows you to fine tune your application in a simulated environment before implementing it in real life. 
Furthermore, ROS has a large community of thousands of users. This allows you to troubleshoot problems and ideas to other people and get assistance with your project. 

Other complementary software tools can be integrated into ROS like **MoveIt** for performing motion planning and manipulation of objects and navigation and **Gazebo**, a physics engine that allows the visualization of your robot.  

# How does ROS work ?
Each piece of code that executes a specific action is called a **node**. For example, Node1 reads in the angular displacement (odometry) of the wheels of a robot and tells you where the robot is, Node2 is a path planner that tells you how to move from point A to B and Node3 is the motor controller which allows you to manipulate each wheel of your robot. A depiction of this situation can be found in the figure below. 
These nodes need to communicate to each other in order to get the robot moving. This communication is done trough **topics** which is a named channel where nodes can send data on. This means that ROS nodes communicate using name based channels. The data that actually is send on the topics are called **messages**. For example, the message send via the /odom topic can be the current position and pose of the robot and the message send via /cmd_vel topic can be the velocity set by the path planner in its quest to get from A to B. 

![](images/nodes_example.png)

These messages are a serialization format for structured data. It allows nodes written in different programming languages (commonly python and C++) to communicate with each other. This structure is defined in a .msg file.
ROS has standard messages which can be used directly but you can also create your own .msg file to define a specific message structure.

ROS uses a **publish/subscribe** message model. This means that a node can log-on (subscribe) to a specific topic name for information and can publish information onto a topic name. Some properties of this model are:
- Any node can publish a message to any topic
- Any node can subscribe to any topic
- Multiple nodes can publish to the same topic
- Multiple nodes can subscribe to the same topic
- A node can publish multiple topics
- A node can subscribe to multiple topics

This allows you to be very flexible and adaptable with communication. But these nodes must be able to find each other in order to communicate with each other. Therefore a ROS master is used

## ROS master
The role of the ROS master is to enable individual ROS nodes to locate one another. It provides naming and registrations services to the nodes in the ROS system. It tracks publishers and subscribers to topics as well as services. Once the nodes have located each other via the ROS master, they communicate with each other peer-to-peer. 
The ROS master is started by using the **roscore** or **roslaunch** command. 

![](images/ros_master.png)

The above image is a rough representation of the communication set up. First off, the ROS master is activated. After that the nodes will register to the master. Once this has happened, the master knows which nodes are publishing on which topics and which nodes want to subscribe to which topic. The master in turn communicates back to the nodes so that a peer-to-peer connection can be established.  

A more detailed explanation is represented by the figure below.

![](images/ros_master_detailed.png)

Once a ROS master is started an address is connected to it. This is hard coded to be localhost:11311.
The nodes need to know the network address of the master on startup to be able to connect to it. This is why the address is hard coded.
When a node is started, a random port number is assigned to it. For example the publisher node will be publishing a topic named /topic on localhost:4321. When the Subscriber node is started, it tells the ROS master that it wants to subscribe to the topic named /topic.
The master in turn tells the subscriber node that this topic is published at localhost:4321. After that the publisher and subscriber establish a connection to exchange data. 
Take note that the master does not handle any information being send. It only starts up the connection between publisher and subscriber. 

Of course this publish/subscribe model is many-to-many one-way transport. This is not appropriate for **request/reply** interactions which is often required in a distributed system. This  is done via a **Service** which is defined by a pair of messages, one for request and one for reply. A node can register a specific service with the ROS master just as it registers/subscribes to its messages. 

For instance, we have a <u>Camera node</u>  which collects data from a camera and we have an <u>Image processing node</u> to process the images. This situation is depicted in the figure below. 
Say that we want to request data from the Camera node at a specific time to process it using the Image processing node.  

![](images/image_processing_example.png)

In this example the Image processing node first sends requests for /image_data. The Camera node collects the data from the camera and then sends a message over /image_data as a reply to the request. 

# Why ROS ?
ROS is certainly not the only robotics middleware available. Other options are:
- Urbi
- MIRO
- YARP
- OpenRDK
- etc.

We will not go into detail on the structure and advantages of every other available robotics middleware compared to ROS. We do not want to diminish their reputation or influence the reader on their choice of robotics middleware. Instead we will focus on the advantages and possibilities ROS has to offer. 

Since nodes are connected via a distributed message system, only the necessary nodes can be called which can deminish the overall program size. This modularity also has the advantage that, if a component crashes, the whole system doesn’t crash. For example, your robot can still continue its actions even if some sensors have died. 

ROS is designed for a Linux operating system, however there is the possibility to install and use ROS on macOS and Microsoft Windows. Although these versions are experimental and will run on-top of the OS subsystem for Linux, they are supported by the ROS community. 

ROS isn’t limited to one programming language. Nodes can be written in some of the most commonly used programming environments. Although C++ is  mostly used, you can also write your scripts in Python, java and LISP. 

The ROS wiki page helps new users to get comfortable with ROS. Several tutorials are available written in C++ as well as Python. However these tutorials can sometimes be to technical or don’t explain enough, the community of ROS can answer all questions and help you on your way.

Through the  ROS community and the fact that ROS is open source, we can avoid the need to develop own code for many aspects of robot control. Troubleshooting via the ROS community is quick and effective. This makes the development of projects faster and easier. 

Many of the other robotics middleware mentioned above are can be integrated into ROS.  The new version of Urbi is compatible with ROS. This means that ROS users can use Urbiscript which provides features like parallelism and event-triggers for creating behavior scripts for robots. 
YARP and ROS can also be integrated with each other. This means that we can use ROS with existing YARP programs. This again allows you to  avoid to develop new scripts for robot control.

# Sources: 
http://www.willowgarage.com/sites/default/files/icraoss09-ROS.pdf
http://wiki.ros.org/ROS/Introduction
http://wiki.ros.org/Installation/Windows
http://wiki.ros.org/kinetic/Installation
http://wiki.ros.org/Master
https://www.youtube.com/watch?v=UL1_Ue4rUWs
http://robohub.org/ros-101-intro-to-the-robot-operating-system/
https://en.wikipedia.org/wiki/Robot_Operating_System
https://www.youtube.com/watch?v=bJB9tv4ThV4
http://www.yarp.it/yarp_with_ros.html
http://www.ros.org/news/2010/07/urbi-open-source-now-integrated-with-ros.html